# stocks notebook

- [Spark2 with Python notebook](https://datascienceplus.com/leveraging-hive-with-spark-using-python/)

In [1]:
!echo $PYTHONPATH

/usr/hdp/current/spark2-client/python:/usr/hdp/current/spark2-client/python/lib/py4j-0.10.6-src.zip


In [8]:
import os
os.listdir(os.getcwd())

['Untitled.ipynb', '.ipynb_checkpoints', 'derby.log', 'metastore_db']

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.enableHiveSupport().getOrCreate()

In [5]:
spark.sql('show databases').show()

+------------+
|databaseName|
+------------+
|     default|
|    foodmart|
|   stocks_db|
+------------+



In [6]:
spark.sql('show tables').show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
+--------+---------+-----------+



In [7]:
spark.sql('use stocks_db')

DataFrame[]

In [8]:
spark.sql('show tables').show()

+---------+---------+-----------+
| database|tableName|isTemporary|
+---------+---------+-----------+
|stocks_db|   stocks|      false|
+---------+---------+-----------+



In [10]:
spark.sql("describe formatted stocks").show(truncate = True)

+--------------------+--------------------+-------+
|            col_name|           data_type|comment|
+--------------------+--------------------+-------+
|                exch|              string|   null|
|              symbol|              string|   null|
|                 ymd|           timestamp|   null|
|          price_open|               float|   null|
|          price_high|               float|   null|
|           price_low|               float|   null|
|         price_close|               float|   null|
|              volume|                 int|   null|
|     price_adj_close|               float|   null|
|                    |                    |       |
|# Detailed Table ...|                    |       |
|            Database|           stocks_db|       |
|               Table|              stocks|       |
|               Owner|           maria_dev|       |
|        Created Time|Sat Jul 10 20:03:...|       |
|         Last Access|Thu Jan 01 00:00:...|       |
|          C

In [11]:
spark.sql("select symbol, count(*) as count from stocks\
          group by symbol\
          having count(*) > 100 \
          order by count desc").show()

+----------+-----+
|    symbol|count|
+----------+-----+
|     TITAN|  992|
|     CIPLA|  992|
|        LT|  992|
|      IDEA|  992|
|TORNTPOWER|  992|
|     L&TFH|  992|
|      GSPL|  992|
|       CUB|  992|
|COROMANDEL|  992|
| ICICIBANK|  992|
|  MINDTREE|  992|
|SRTRANSFIN|  992|
|IBULHSGFIN|  992|
|MANAPPURAM|  992|
|  NIFTY 50|  992|
|      SAIL|  992|
|  DIVISLAB|  992|
|   SYNGENE|  992|
|  EXIDEIND|  992|
|  HDFCBANK|  992|
+----------+-----+
only showing top 20 rows

